In [22]:
import random
from Bio import SeqIO
import re

def read_cluster_file(cluster_filename):
    cluster_dict = {}
    with open(cluster_filename, "r") as f:
        for line in f:
            if line.startswith(">"):
                cluster_name = line.strip()
                cluster_dict[cluster_name] = []
            else:
                gene_name = line.split(",")[1].strip().split("...")[0]
                is_ref = "*" in line
                cluster_dict[cluster_name].append((gene_name, is_ref))
    return cluster_dict

def choose_gene(cluster_dict):
    selected_genes_vf = {}
    selected_genes_deg = {}

    for cluster, genes in cluster_dict.items():
        vf_genes = [gene for gene in genes if not gene[0].startswith(">DEG")]
        deg_genes = [gene for gene in genes if gene[0].startswith(">DEG")]

        if vf_genes:  # if there's any VF gene, choose one VF gene
            selected_genes_vf[cluster] = random.choice(vf_genes)
        elif deg_genes:  # if there's only DEG genes, choose a DEG gene
            selected_genes_deg[cluster] = random.choice(deg_genes)

    return selected_genes_vf, selected_genes_deg

def extract_sequences(fasta_file, selected_genes, output_file):
    sequences = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

    with open(output_file, "w") as f:
        for genes in selected_genes.values():
            gene_name = genes[0].strip(' >') # remove ">" symbol from the gene name
            if gene_name in sequences:
                SeqIO.write(sequences[gene_name], f, "fasta")
            else:
                print(f"Warning: {gene_name} not found in the fasta file.")
            
            

cluster_filename = "All_data_1.0.clstr"
fasta_file_all_vf = "VF/All_VF.fasta"
fasta_file_deg = "N_VF/Essential_genes.fasta"
output_file_vf = "VF/F_VF.fasta"
output_file_deg = "N_VF/F_DEG.fasta"

cluster_dict = read_cluster_file(cluster_filename)
selected_genes_vf, selected_genes_deg = choose_gene(cluster_dict)

extract_sequences(fasta_file_all_vf, selected_genes_vf, output_file_vf)
extract_sequences(fasta_file_deg, selected_genes_deg, output_file_deg)


In [24]:
def count_sequences(fasta_file):
    return sum(1 for _ in SeqIO.parse(fasta_file, "fasta"))

fasta_file = "N_VF/F_DEG.fasta"
num_sequences = count_sequences(fasta_file)

print(f"There are {num_sequences} sequences in the DEG file.")

There are 22024 sequences in the fasta file.


In [28]:
def count_sequences(fasta_file):
    return sum(1 for _ in SeqIO.parse(fasta_file, "fasta"))

fasta_file = "VF/F_VF.fasta"
num_sequences = count_sequences(fasta_file)

print(f"There are {num_sequences} sequences in the VF file.")

There are 11688 sequences in the fasta file.
